# DICE Model (at home)
**Exercise Session Resource Economics (Spring Term 2025)** \
Raul Hochuli (raul.hochuli@unibas.ch)

This is the first part of teaching the build up of the DICE model. **Please try to complete the Tasks in the `Code` section of the notebook before class on Monday, 7th April 2025**. I ask you once again to email my any solutions you have until Sunday evening before the class, so I can try to adjust my teaching accordingly. 

### Preamble


The following code has been replicated and refactored for this course based on the inital material from [Hazem Krichene](https://github.com/hazem2410) in his public Github repository [PYDICE](https://github.com/hazem2410/PyDICE), which builds on the [DICE (Dynamic Integrated Climate Economy) Model of William D. Nordhaus](https://williamnordhaus.com/dicerice-models). 

### Naming Convention Table

| **Lecture**     | **Exercise Session**              | **Default Values**     | **Description**                                                                 |
|-----------------|-----------------------------------|------------------------|---------------------------------------------------------------------------------|
| $T$             | `T_end`                           | 100                    | Number of periods                                                               |
|                 | `tstep`                           | 5                      | Time step size                                                                  |
|                 | `scale1`                          | 0.0302455265681763     | Multiplicative scaling coefficient                                              |
|                 | `scale2`                          | -10993.704             | Additive scaling coefficient                                                    |
| $A$             | `TFPr`                            | exogenous              | Total factor productivity                                                       |
| $L$             | `Labr`                            | exogenous              | World population (millions, 2015: 7403; 2050: 11500)                            |
| $\sigma$        | `Dcrb`                            | exogenous              | Decarbonization rate (GTC / trillion 2010 USD)                                  |
| $\theta_1$      | `cost1_Abat`                      | exogenous              | Abatement cost multiplicative term                                              |
| $E_{Land}$      | `CEms_land`                       | exogenous              | Carbon emissions through land use (deforestation), GtC                          |
| $S$             | `Savi`                            | decision variable      | Savings rate                                                                    |
| $\mu$           | `ECtr`                            | decision variable      | Emission control rate                                                           |
| $\alpha$        | `elast_mrg_cons_utility`          | 1.45                   | Elasticity of marginal utility of consumption                                   |
| $\gamma$        | `elast_capt_in_prod`              | 0.300                  | Elasticity of capital in production function                                    |
| $\rho$          | `social_time_pref_rate`           | 0.015                  | Initial rate of social time preference per year                                 |
|                 | `depr_Capt`                       | 0.1                    | Depreciation rate on capital (per year)                                         |
| $K$             | `Capt0`                           | 223.3                  | Initial world capital 2015 (trillion 2010 USD)                                  |
| $F$             | `ncrb_RFor0`                      | 0.5                    | 2015 forcings of non-CO2 GHG (Wm-2)                                             |
|                 | `ncrb_RFor1`                      | 1.0                    | 2100 forcings of non-CO2 GHG (Wm-2)                                             |
|                 | `t_ncrb_RFor`                     | 18                     | Period when non-CO2 forcings switch                                             |
|                 | `incr_RFor_dbl_crbn`              | 3.6813                 | Forcing from doubling CO2 (Wm-2) 2015                                           |
|                 | `incr_temp_dbl_crb`               | 3.1                    | Equilibrium temperature impact (°C per doubling CO2)                            |
| $\pi_1$         | `a1`                              | 0                      | Damage function intercept                                                       |
| $\pi_2$         | `a2`                              | 0.00236                | Damage function quadratic term                                                  |
|                 | `a3`                              | 2.00                   | Damage function exponent                                                        |
| $\theta_2$      | `cost2_Abat`                      | 2.6                    | Exponent of control cost function                                               |
|                 | `decl_Bstp`                       | 0.025                  | Initial cost decline backstop cost per period                                   |
|                 | `Bstp0`                           | 550                    | Cost of backstop technology 2010 USD per ton CO2                                |
| $\mu_0$         | `ECtr0`                           | 0.03                   | Initial emissions control rate 2015                                             |
|                 | `ECtr_lim`                        | 1.2                    | Upper limit on control rate after 2150                                          |
| $M_{AT}$        | `CStk_atmo0`                      | 851                    | Initial carbon stock in atmosphere 2015 (GtC)                                   |
| $M_{AT}(1750)$  | `CStk_atmo1750`                   | 588                    | Initial carbon stock in 1750                                                    |
| $M_{UP}$        | `CStk_ocup0`                      | 460                    | Initial carbon stock in upper strata 2015 (GtC)                                 |
| $M_{LO}$        | `CStk_oclo0`                      | 1740                   | Initial carbon stock in lower strata 2015 (GtC)                                 |
|                 | `CStk_atmo_eq`                    | 588                    | Equilibrium carbon stock atmosphere (GtC)                                       |
|                 | `CStk_ocup_eq`                    | 360                    | Equilibrium carbon stock upper strata (GtC)                                     |
|                 | `CStk_oclo_eq`                    | 1720                   | Equilibrium carbon stock lower strata (GtC)                                     |
| $\phi_{12}$     | `b12`                             | 0.12                   | Carbon cycle transition matrix                                                  |
| $\phi_{23}$     | `b23`                             | 0.007                  | Carbon cycle transition matrix                                                  |
| $T_{AT}$        | `Temp_atmo0`                      | 0.85                   | Initial atmospheric temperature change (°C from 1900)                           |
| $T_{LO}$        | `Temp_ocea0`                      | 0.0068                 | Initial lower ocean temperature change (°C from 1900)                           |
| $\xi_1$         | `c1`                              | 0.1005                 | Transfer coefficient climate equation atmoshpere temperature                    |
| $\xi_3$         | `c3`                              | 0.088                  | Transfer coefficient atmo-ocen difference                                       |
| $\xi_4$         | `c4`                              | 0.025                  | Transfer coefficient for lower ocean level                                      |

In [ ]:
# Packages used in the notebook
import os as os
import numpy as np
import pandas as pd
import glob
import datetime as dt

import plotly.graph_objects as go

from plotly.subplots import make_subplots

### Building up the Economic Part of the Model


Before diving into an acutal optimization of the DICE model, we first want to build the dynamics for the ecnomic part of the model. This means we build (almost) all the functions that define the interplay between state variables. We then look at this interplay without any optimizations but just observe how constant exogenous variables translate down into endogenous state variables. For this, try to complete the tasks below (indicated as `<..>` comments in the code) and refine the economic part of the model step by step. 

Note: 
- The visualization of the results gives little information for now, but shows that your code runs through 
- Exogenous Variables are variables based purely on assumptions and are not affected by any model changes. For now we consider no changes in these variables and assume they are just constant over time. 

Tasks: 
1. Use the social discount rate `social_time_pref_rate` to make an social discount array `social_time_pref_array` $\frac{1}{(1+\rho)^t}$ to be later applied in the calculations. Watch out that $t$ is denoted in 5 years steps. 
2. Define the exogenous variables for total factor productivity `TFPr`and decarbonization rate `Dcrb` as constants of the initial value (`TFPr0`, `Dcrb0`)
3. Complete the discounted utility `fUtil_cons_disc()` (welfare function $W_t$ in the lecture). 

Run all the functions (the entire cell for a first time, it should give a first graph output)

4. Add a capital depreciation rate `depr_Capt` of 0.1 to the function `fCapt()`defining the capital stock
5. Add a constant savings rate of $S_t = 0.08$ for all $t$ to the model such that investments are possible and the entire production is not just consumed. 


In [8]:
# PARAMETERS --------------------------------------------------------------------------------
T_end   = 100
tstep   = 5
t = np.arange(1, T_end + 1)
NT = len(t)


# elasticities
elast_mrg_cons_utility = 1.45
elast_capt_in_prod     = 0.300

# discounting
social_time_pref_rate  = 0.015
social_time_pref_array = 1/((1+social_time_pref_rate)**(tstep*(t-1))) # <.1.>



# EXOGENOUS VARS -------------------------------------------------------------------------------------------
# labour (Labr) 
Labr0 = 7403    # inital world population (millions) 2015
Labr = np.full(NT, Labr0)

# total factor productivity (TFP)
TFPr0 = 5.115   # initial level of total factor productivity 2015
TFPr = np.full(NT, TFPr0)  # <.2.>

# decarbonization rate (Dcrb)
Dcrb0 = 0.35032
Dcrb = np.full(NT, Dcrb0)   # <.2.>

# carbon land emissions (CEms_land)
CEms_land0 = 2.6   # initial carbon emissions from land use 2015, (2.6 GtCO2/yr)
CEms_land = np.full(NT, CEms_land0)   

# abatement cost (cost1_Abat)
Bstp0 = 550  # Cost of backstop technology 2010$ per ton CO2
cost1_Abat = np.full(NT, Bstp0)  



# ENDOGENOUS VARS -------------------------------------------------------------------------------------------

# utility
Util_pcap_cons = np.zeros(NT)
def fUtil_pcap_cons(iCons, iLabr, idx):
    return ((iCons[idx] * 1000 / iLabr[idx]) ** (1 - elast_mrg_cons_utility) -1) / (1 - elast_mrg_cons_utility) - 1  

Util_cons_disc = np.zeros(NT)
def fUtil_cons_disc(iUtil_pcap_cons, iLabr, idx):
    return iUtil_pcap_cons[idx] * iLabr[idx] * social_time_pref_array[idx]   # <.3.> 


# consumption 
Cons = np.zeros(NT)
def fCons(iProd_gross, iInvs, idx):
    return iProd_gross[idx] - iInvs[idx]


Cons_pcap = np.zeros(NT)
def fCons_pcap(iCons, iLabr, idx):
    return 1000 * iCons[idx] / iLabr[idx]


# production
Prod_gross = np.zeros(NT)
def fProd_gross(iTFPr, iLabr, iCapt, idx):
    return iTFPr[idx] * ((iLabr[idx]/1000) ** (1 - elast_capt_in_prod)) * (iCapt[idx] ** elast_capt_in_prod)


# capital
Capt0           = 223.3
depr_Capt       = 0.1     # <.4.>
Capt            = np.zeros(NT)
Capt[0]         = Capt0
def fCapt(iCapt, iInvs, idx):
    if (idx == 0):
        return Capt0
    else:
        return (1-depr_Capt)**tstep * iCapt[idx-1] + tstep * iInvs[idx-1]   # <.4.>


# investment 
Invs = np.zeros(NT)
Savi = np.full(NT, 0.08) # <.5.> 
def fInvs(iSavi, iProd_gross, idx):
    return iSavi[idx] * iProd_gross[idx] # <.5.>



# UPDATE STATE VARIABLES over time ------------------------------------------------------------ 
for i in range(NT):
    Capt[i] = fCapt(Capt, Invs, i)
    Prod_gross[i] = fProd_gross(TFPr, Labr, Capt, i)
    Invs[i] = fInvs(Savi, Prod_gross, i)
    Cons[i] = fCons(Prod_gross, Invs, i)
    Cons_pcap[i] = fCons_pcap(Cons, Labr, i)
    Util_pcap_cons[i] = fUtil_pcap_cons(Cons, Labr, i)
    Util_cons_disc[i] = fUtil_cons_disc(Util_pcap_cons, Labr, i)



# EXPORT RESULTS ------------------------------------------------------------
tuples_to_plot = (
    (Util_pcap_cons,        'Util_pcap_cons'),   # <..>
    (Util_cons_disc,        'Util_cons_disc'), 
    (Cons,                  'Cons'), 
    (Cons_pcap,             'Cons_pcap'), 
    (Prod_gross,            'Prod_gross'), 
    (Capt,                  'Capt'), 
    (Invs,                  'Invs'), 
)


results_df = pd.DataFrame({'t': t, 't_year': 2000 + t*tstep})
for var_object, col_name in tuples_to_plot:
    results_df[col_name] = var_object



# # VISUALIZATION ------------------------------------------------------------
cols_to_plot = [col for col in results_df.columns if col not in ['t', 't_year']]
n_rows = len(cols_to_plot) // 3 + 1 if len(cols_to_plot) % 3 > 0 else len(cols_to_plot) // 3
fig = make_subplots(rows=n_rows, cols =3, subplot_titles=cols_to_plot)
for i, col in enumerate(cols_to_plot):
    row_idx = i // 3 + 1
    col_idx = i % 3 + 1
    fig.add_trace(go.Scatter(x=results_df['t_year'], y=results_df[col], name=col), row=row_idx, col=col_idx)
    fig.update_xaxes(title_text="Year", row=row_idx, col=col_idx)

fig.update_layout(title_text="Economic State variables", template = 'plotly_white', )
fig.show()




### Building up the Emission Part of the Model

After having completed the economic part of the DICE Model, let's consider the emission part. The emission part has more complex functional terms defining its state variables but follows exaclty the same structure as the economic part. Try to complete the tasks below (indicated as `<..>` comments in the code) and refine the emission part of the model step by step. 

Note:
- `Prod_gross` is here the state variables direclty influencing the amount of emissions `CEms` which trickles down through the emission part of the model. If you were able to solve the emission part, use the values for `Prod_gross` you got there, if not just use a flat constant given below. 

Tasks: 
1. Build the function `fDamg_frac()` defining the fractional damages of the atmosphere temperature `iTemp_atmo` ($a_1 \cdot T_{AT,t} + a_2 \cdot T_{AT,t}^{a_3}$)
2. Complete the missing elements of the carbon cycle transition matrix (hint 1: $b_{11} = 1-b_{12}$; hint 2: $b_{32} = b_{23}(M_{LO}^{*}/M_{UP}^{*})$)
3. Complete the function `fCStk_atmo()` defining the carbon stock in the atmosphere
4. Complete the function `fCStk_ocup()` defining the carbon stock in the upper ocean layer
5. Add an constant emission control rate `ECtr` of 0.2 to the model and rerun the state variable definitions. 

In [ ]:
# PARAMETERS --------------------------------------------------------------------------------
T_end   = 100
tstep   = 5
t = np.arange(1, T_end + 1)
NT = len(t)


# GET PRODUCTION VARS -------------------------------------------------------------------------------------------
Prod_gross = np.full(NT, 108)
# OR 
# Prod_gross = results_df['Prod_gross'] 



# EXOGENOUS VARS -------------------------------------------------------------------------------------------
# labour (Labr) 
Labr0 = 7403    # inital world population (millions) 2015
Labr = np.full(NT, Labr0)

# total factor productivity (TFP)
TFPr0 = 5.115   # initial level of total factor productivity 2015
TFPr = np.full(NT, TFPr0)  

# decarbonization rate (Dcrb)
Dcrb0 = 0.35032
Dcrb = np.full(NT, Dcrb0)  

# carbon land emissions (CEms_land)
CEms_land0 = 2.6   # initial carbon emissions from land use 2015, (2.6 GtCO2/yr)
CEms_land = np.full(NT, CEms_land0)   

# abatement cost (cost1_Abat)
Bstp0 = 550  # Cost of backstop technology 2010$ per ton CO2
cost1_Abat = np.full(NT, Bstp0)  


# ENDOGENOUS VARS -------------------------------------------------------------------------------------------
ECtr = np.zeros(NT) # # <.5.>

# carbon emissions (industry and total)
CEms_indu = np.zeros(NT)
def fCEms_indu(iProd_gross, iECtr, iDcrb, idx):
    return iDcrb[idx] * iProd_gross[idx] * (1 - iECtr[idx])

CEms = np.zeros(NT)
def fCEms(iCEms_indu, iCEms_land, idx):
    return iCEms_indu[idx] + iCEms_land[idx]


# radiative forcing (RFor)
incr_RFor_dbl_crbn = 3.6813                   # Forcing from doubling CO2 (Wm-2) 2015
incr_temp_dbl_crb = 3.1                      # Equilibrium temp impact (oC per doubling CO2)
CStk_atmo0    =  851                      # Initial Concentration in atmosphere 2015 (GtC)
CStk_atmo1750 =  588                      # Initial Concentration in 1750, 

RFor = np.zeros(NT)
def fRFor(iCStk_atmo, idx):
    return incr_RFor_dbl_crbn * np.log(iCStk_atmo[idx] / CStk_atmo1750) / np.log(2) 

# damages
a1 = 0                       # Damage function intercept
a2 = 0.00236               # Damage function quadratic term
a3 = 2.00                  # Damage function exponent

Damg_frac = np.zeros(NT)
def fDamg_frac(iTemp_atmo, idx):
    return a1 * iTemp_atmo[idx] + a2*iTemp_atmo[idx]**a3   # <.1.>

Damg = np.zeros(NT)
def fDamg(iProd_gross, iDamg_frac, idx):
    return iProd_gross[idx] * iDamg_frac[idx]


# abatement cost
cost2_Abat = 2.6                    

Abat_cost = np.zeros(NT)
def fAbat_cost(iProd_gross, iECtr, icost1_Abat, idx):
    return iProd_gross[idx] * icost1_Abat[idx] * iECtr[idx] ** cost2_Abat

Abat_MC = np.zeros(NT)
Bstp = np.full(NT, Bstp0)  
def fAbat_MC(iECtr, idx):
    return Bstp[idx] * iECtr[idx] ** (cost2_Abat-1)


# carbon cycle ocean transition matrix
CStk_ocup0    =  460                      # Initial Concentration in upper strata 2015 (GtC)
CStk_oclo0    =  1740                     # Initial Concentration in lower strata 2015 (GtC)
CStk_atmo_eq  =  588                      # Equilibrium concentration atmosphere (GtC)
CStk_ocup_eq  =  360                      # Equilibrium concentration upper strata (GtC)
CStk_oclo_eq  =  1720                     # Equilibrium concentration lower strata (GtC)
b12           =  0.12                     # carbon cycle transition matrix (atmo to up)
b23           =  0.007                    # carbon cycle transition matrix (up to lo)

b11 = 1 - b12  # <.2.>
b21 = b12*(CStk_ocup_eq/CStk_atmo_eq)   
b22 = 1 - b21 - b23
b32 = b23*(CStk_oclo_eq/CStk_ocup_eq) # <.2.>
b33 = 1 - b32

CStk_atmo       = np.zeros(NT)
CStk_atmo[0]    = CStk_atmo0
def fCStk_atmo(iCStk_atmo, iECtr, iCEms, idx):
    if (idx == 0):
        return CStk_atmo0
    else:
        return iCStk_atmo[idx-1]*b11 + CStk_ocup[idx-1]*b21 + iCEms[idx-1]*5/3.666    # <.3.>

CStk_ocup       = np.zeros(NT)
CStk_ocup[0]    = CStk_ocup0
def fCStk_ocup(iCStk_atmo, iCStk_ocup, iCStk_oclo, idx):
    if (idx == 0):
        return CStk_ocup0
    else:
        return iCStk_atmo[idx-1]*b12 + iCStk_ocup[idx-1]*b22 + iCStk_oclo[idx-1]*b32   # <.4.>

CStk_oclo       = np.zeros(NT)
CStk_oclo[0]    = CStk_oclo0
def fCStk_oclo(iCStk_ocup, iCStk_oclo, idx):
    if (idx == 0):
        return CStk_oclo0
    else:
        return iCStk_oclo[idx-1]*b33 + iCStk_ocup[idx-1]*b23
    

# temperature
Temp_atmo0 = 0.85                    # Initial atmospheric temp change (C from 1900)
Temp_ocea0 = 0.0068                   # Initial lower stratum temp change (C from 1900)
c1         = 0.1005                   # Climate equation coefficient for upper level
c3         = 0.088                    # Transfer coefficient upper to lower stratum
c4         = 0.025                    # Transfer coefficient for lower level

Temp_atmo       = np.zeros(NT)
Temp_atmo[0]    = Temp_atmo0
def fTemp_atmo(iTemp_atmo, iRFor, iTemp_ocea, idx):
    if (idx == 0):
        return Temp_atmo0
    else:
        return iTemp_atmo[idx-1] + c1*(iRFor[idx] - (incr_RFor_dbl_crbn/incr_temp_dbl_crb)*iTemp_atmo[idx-1] - c3*(iTemp_atmo[idx-1] - iTemp_ocea[idx-1])) 

Temp_ocea       = np.zeros(NT)
Temp_ocea[0]    = Temp_ocea0
def fTemp_ocea(iTemp_atmo, iTemp_ocea, idx):
    if (idx == 0):
        return Temp_ocea0
    else:
        return iTemp_ocea[idx-1] + c4*(iTemp_atmo[idx-1] - iTemp_ocea[idx-1])


# UPDATE STATE VARIABLES over time ------------------------------------------------------------ 
for i in range(NT): 
    CEms_indu[i] = fCEms_indu(Prod_gross, ECtr, Dcrb, i)
    CEms[i]      = fCEms(CEms_indu, CEms_land, i)
    RFor[i]      = fRFor(CStk_atmo, i)
    Damg_frac[i] = fDamg_frac(RFor, i)
    Damg[i]      = fDamg(Prod_gross, Damg_frac, i)
    Abat_cost[i] = fAbat_cost(Prod_gross, ECtr, cost1_Abat, i)
    Abat_MC[i]   = fAbat_MC(ECtr, i)
    CStk_atmo[i] = fCStk_atmo(CStk_atmo, ECtr, CEms, i)
    CStk_ocup[i] = fCStk_ocup(CStk_atmo, CStk_ocup, CStk_oclo, i)
    CStk_oclo[i] = fCStk_oclo(CStk_ocup, CStk_oclo, i)
    Temp_atmo[i] = fTemp_atmo(Temp_atmo, RFor, Temp_ocea, i)
    Temp_ocea[i] = fTemp_ocea(Temp_atmo, Temp_ocea, i)



# EXPORT RESULTS ------------------------------------------------------------
tuples_to_plot = (
    (CEms_indu,         'CEms_indu'),          
    (CEms,              'CEms'),       
    (RFor,              'RFor'),       
    (Damg_frac,         'Damg_frac'),          
    (Damg,              'Damg'),       
    (Abat_cost,         'Abat_cost'),          
    (Abat_MC,           'Abat_MC'),        
    (CStk_atmo,         'CStk_atmo'),          
    (CStk_ocup,         'CStk_ocup'),          
    (CStk_oclo,         'CStk_oclo'),          
    (Temp_atmo,         'Temp_atmo'),          
    (Temp_ocea,         'Temp_ocea'),          
)

results_df = pd.DataFrame({'t': t, 't_year': 2000 + t*tstep})
for var_object, col_name in tuples_to_plot:
    results_df[col_name] = var_object



# # VISUALIZATION ------------------------------------------------------------
cols_to_plot = [col for col in results_df.columns if col not in ['t', 't_year']]
n_rows = len(cols_to_plot) // 3 + 1 if len(cols_to_plot) % 3 > 0 else len(cols_to_plot) // 3
fig = make_subplots(rows=n_rows, cols =3, subplot_titles=cols_to_plot)
for i, col in enumerate(cols_to_plot):
    row_idx = i // 3 + 1
    col_idx = i % 3 + 1
    fig.add_trace(go.Scatter(x=results_df['t_year'], y=results_df[col], name=col), row=row_idx, col=col_idx)
    fig.update_xaxes(title_text="Year", row=row_idx, col=col_idx)

fig.update_layout(title_text="Economic State variables", template = 'plotly_white', )
fig.show()


C:\Users\hocrau00\AppData\Local\Temp\ipykernel_11404\4287919837.py:58: RuntimeWarning:

divide by zero encountered in log

C:\Users\hocrau00\AppData\Local\Temp\ipykernel_11404\4287919837.py:67: RuntimeWarning:

invalid value encountered in scalar multiply

C:\Users\hocrau00\AppData\Local\Temp\ipykernel_11404\4287919837.py:140: RuntimeWarning:

invalid value encountered in scalar subtract

